In [ ]:
%autosave 0

In [ ]:
import subprocess
from os import listdir
from os.path import isfile, join
from time import time, sleep

# Tagfuse File System Feature Test

This file contains a set of tests that verify the features of the Tag Fuse Filesystem.

## Prerequisites

- Raspberry Pi with latest software (See ```01_RPi_Build.md``` for details on contructing and installing a new RPi Disk Image). Make sure that the lates TagNet Basestation Software is installed, too.
- Tag with latest software (see ```?``` for details on constructing and installing a new Tag Image)

### Verified Features
- Polling for tags
    - tested with two
- Dblk read
    - using ```tagdump```
- Dlbk special files
    - sizes reflect offsets of underlying record status

### To be Tested
- Panic verification
    - Panic byte file read is working
    - need to figure out if/how to erase
- GPS XYZ value
- GPS cmd
    - has been used extensively, just need to add test
- Poll count verification
- Reboot using sys/{Active,Golden,Nib,Backup,Running}

### Failed Tests

- getattr
    - rtc
    - panic/{count, byte}
    - test/{sum,zeros,ones}
    - sys/{golden,nib}

### Missing Features
- Image filesize and timestamp metadata
- Not all file timestamps are being set
- Poll event identifiers
- Date on GPS (designates acquisition time)

## Initial State

The following variables provide user environment control.

In [ ]:
nid='658bc8e5205c'
#nid='a0b696b4fb29'

In [ ]:
TREE_BASE       = '/home/pi/tags'
TAG_ID          = TREE_BASE + '/' + nid
TAG_BASE        = TAG_ID + '/tag'
TAG_IMAGE_STORE = TAG_BASE + '/sd/0/img'
DBLK_BASE       = TAG_BASE + '/sd/0/dblk'
SYS_BASE        = TAG_BASE + '/sys'
POLL_BASE       = TAG_BASE + '/poll'
SPARSE_STORE    = '/home/pi/Desktop'
BS_IMAGE_STORE  = '/mnt/neptune/tagbin'

## NOTES / ISSUES
- writing image when no room available gets stuck in a loop - very bad

## Install Tagfuse Driver

First, get the tagfuse driver running as a daemon. TagFuse can be started separately, just comment out these invocation below.

In [ ]:
#!python ../tagfuse/tagfuse -b -s $SPARSE_STORE $TREE_BASE
#!tagfuse -b -s $SPARSE_STORE $TREE_BASE

Now look at the entire tree of information available in the tag

In [ ]:
!tree -a $TAG_ID
#!tree -aJsD $TREE_BASE

## Poll for Tags

In [ ]:
!ls $TREE_BASE/.poll

In [ ]:
!ls -al $TREE_BASE

## Examine the Dblk Area

In [ ]:
#STOP

In [ ]:
!ls -al $DBLK_BASE

In [ ]:
!echo `stat -c '%s' $DBLK_BASE/.last_rec`

In [ ]:
!tagdump -n 10 --net -j `stat -c '%s' $DBLK_BASE/.last_sync` $DBLK_BASE/byte

In [ ]:
# !tagdump -j `stat -c '%s' $DBLK_BASE/.last_rec` $DBLK_BASE/byte

## Write a note into the Dblk Area

In [ ]:
!echo 'this is a note' > $DBLK_BASE/note

In [ ]:
ls -l $DBLK_BASE/note

In [ ]:
!echo 'this is a second note' > $DBLK_BASE/note

In [ ]:
ls -l $DBLK_BASE/note

## Extract a Panic Event

## Get GPS Position

## Send Commands to control the GPS

## Examine Pending Events on the Tag

In [ ]:
!ls -l $POLL_BASE

## Show the state of software currently installed on the tag

In [ ]:
!tree $SYS_BASE

## Show the currently active version

In [ ]:
!ls -l $SYS_BASE/active

## Show list of Images stored on Tag

In [ ]:
!ls -l $TAG_IMAGE_STORE

## Show list of Images stored on the Basestation

In [ ]:
!ls -l $BS_IMAGE_STORE

In [ ]:
cmd='ls ' + BS_IMAGE_STORE + '/*'
basestation_image_list= [ f for f in subprocess.check_output(cmd, shell=True).split('\n') if f]
basestation_image_list

## Exercise the Image Storage and Activation Logic

Perform the following steps:
1. Get Tag's currently active version and save it as original_vers
2. Get list of available images to load (stored on basestation)
3. Remove all images from Tag (should't be able to remove active image)
4. Load three of the available images
5. Set first of the newly loaded images as Active
5. Set second of the newly loaded images as Backup
5. Set third of the newly loaded images as Active
6. Set orignal_vers image as Active

### 1. Get currently active version

In [ ]:
cmd='ls '+SYS_BASE+'/active'
original_vers=subprocess.check_output(cmd, shell=True)[:-1]
original_vers

### 2. Get list of available images to load (stored on basestation)

In [ ]:
basestation_image_list = [f for f in listdir(BS_IMAGE_STORE) if isfile(join(BS_IMAGE_STORE, f))
                          and not f.startswith('.') and not f == original_vers]
basestation_image_list

### 3. Remove all images from Tag
Note: should't be able to remove active image.

In [ ]:
!rm $TAG_IMAGE_STORE/*
!ls -l $TAG_IMAGE_STORE

### 4. Load three of the available images

Use `cp` to copy one of the software images in the IMAGE_STORE to the tag
Monitor progress using pv

In [ ]:
import os
import struct as pystruct
IMAGE_INFO_SIG = 0x33275401
IMAGE_META_OFFSET = 0x140
IMAGE_INFO_DEFAULT = [IMAGE_INFO_SIG, 0x20000, (0x140*2)+0x1c, 
                      9999, 2, 0, 
                      0, 
                      99, 77, 
                      '\00'  * 10]
IMAGE_INFO_LEN = 2
#
# Struct created for accessing image info (little indian)
# sig, image_start, imagelength, vector_chk, image_chk, im_build, im_minor, im_major, main_tree, aux_tree, build_time, im_rev, im_model = image_info
#
IMB_FIELDS = '<LLLHBBLBB10s'
image_info_struct = pystruct.Struct(IMB_FIELDS)
IMAGE_MIN_SIZE  =  (IMAGE_META_OFFSET + image_info_struct.size)

In [ ]:
def info_check(filename):
    with open(filename, 'rb') as infile:
        if not infile:
            return (NONE, NONE, NONE)
        infile.seek(0, 2) # seek to the end
        file_size = infile.tell()
        if file_size < IMAGE_MIN_SIZE: raise RadioLoadException("input file too short")
        infile.seek(0, 0)    # seek to the beginnnig
        # get image info from input file and sanity check
        infile.seek(IMAGE_META_OFFSET) # seek to location of image info
        image_info = image_info_struct.unpack(infile.read(image_info_struct.size))
        print("file information")
        sig, image_start, imagelength, im_build, im_minor, im_major, image_chk, \
             im_rev, im_model, pad = image_info
        pstr = "  signature: 0x{:x}, start: 0x{:x}, length: 0x{:x}, image_chk: 0x{:x}"
        print(pstr.format(sig, image_start, imagelength, image_chk))
        pstr = "  version: ({}.{}.{}(0x{:x})), rev: {}, model: {}"
        print(pstr.format(im_major, im_minor, im_build, im_build, im_rev, im_model))
        if sig != IMAGE_INFO_SIG: raise RadioLoadException("image metadata is invalid")
    return ((im_major, im_minor, im_build), imagelength)

In [ ]:
for i in range(3 if len(basestation_image_list) >= 3 
               else len(basestation_image_list)):
    version = basestation_image_list[i]
    print(version)
    info_check(os.path.join(BS_IMAGE_STORE, version))
    !dd if=$BS_IMAGE_STORE/$version of=$TAG_IMAGE_STORE/$version status=progress

### 5. Set first of the newly loaded images as Active

Set the active version NOTE: REBOOTS TAG

In [ ]:
version=basestation_image_list[0]
!ln $TAG_IMAGE_STORE/$version $SYS_BASE/active/$version

In [ ]:
sleep(5)

In [ ]:
!ls -l $SYS_BASE/active

Try again with same version should fail

In [ ]:
version=basestation_image_list[2]
!ln $TAG_IMAGE_STORE/$version $SYS_BASE/active/$version

### 5. Set second of the newly loaded images as Backup

In [ ]:
!ls -l $SYS_BASE/backup

In [ ]:
version=basestation_image_list[1]
!ln $TAG_IMAGE_STORE/$version $SYS_BASE/backup/$version

In [ ]:
sleep(5)

In [ ]:
!ls -l $SYS_BASE/backup

### 5. Set third of the newly loaded images as Active

In [ ]:
version=basestation_image_list[2]
!ln $TAG_IMAGE_STORE/$version $SYS_BASE/active/$version

In [ ]:
sleep(5)

In [ ]:
!ls -l $SYS_BASE/active

### 6. Set orignal_vers image as Active

Set the backup version

In [ ]:
!ln $TAG_IMAGE_STORE/$original_vers $SYS_BASE/active/$original_vers

In [ ]:
!tree $SYS_BASE

In [ ]:
!ls -l $TAG_IMAGE_STORE

Reboot into the NIB

Reboot into Golden

## Stop the Fuse Driver

In [ ]:
STOP

In [ ]:
!fusermount -u ~/tags